In [1]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [2]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
3718,./AudioWAV/1046_IWW_DIS_XX.wav,disgust.wav
968,./AudioWAV/1012_WSI_HAP_XX.wav,happy.wav
939,./AudioWAV/1012_IWW_NEU_XX.wav,neutral.wav
4752,./AudioWAV/1059_IEO_HAP_HI.wav,happy.wav
3042,./AudioWAV/1038_IOM_SAD_XX.wav,sad.wav


In [3]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [4]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [5]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [6]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [7]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=500):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(sdnn_model.parameters(), lr=0.001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/500], Loss: 6.6548, Accuracy: 0.1697
Epoch [2/500], Loss: 3.6167, Accuracy: 0.1678
Epoch [3/500], Loss: 2.7739, Accuracy: 0.1663
Epoch [4/500], Loss: 2.3615, Accuracy: 0.1757
Epoch [5/500], Loss: 2.1861, Accuracy: 0.1705
Epoch [6/500], Loss: 2.0788, Accuracy: 0.1722
Epoch [7/500], Loss: 2.0399, Accuracy: 0.1698
Epoch [8/500], Loss: 2.0074, Accuracy: 0.1660
Epoch [9/500], Loss: 1.9552, Accuracy: 0.1779
Epoch [10/500], Loss: 1.9438, Accuracy: 0.1646
Epoch [11/500], Loss: 1.9134, Accuracy: 0.1744
Epoch [12/500], Loss: 1.9107, Accuracy: 0.1692
Epoch [13/500], Loss: 1.9018, Accuracy: 0.1640
Epoch [14/500], Loss: 1.8810, Accuracy: 0.1660
Epoch [15/500], Loss: 1.8778, Accuracy: 0.1713
Epoch [16/500], Loss: 1.8693, Accuracy: 0.1745
Epoch [17/500], Loss: 1.8697, Accuracy: 0.1737
Epoch [18/500], Loss: 1.8527, Accuracy: 0.1697
Epoch [19/500], Loss: 1.8488, Accuracy: 0.1789
Epoch [20/500], Loss: 1.8393, Accuracy: 0.1762
Epoch [21/500], Loss: 1.8511, Accuracy: 0.1744
Epoch [22/500], Loss: 

In [10]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(lstm_model.parameters(), lr=0.001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.7774, Accuracy: 0.2316
Epoch [2/500], Loss: 1.7418, Accuracy: 0.2636
Epoch [3/500], Loss: 1.7033, Accuracy: 0.3133
Epoch [4/500], Loss: 1.6691, Accuracy: 0.3128
Epoch [5/500], Loss: 1.6460, Accuracy: 0.3114
Epoch [6/500], Loss: 1.6289, Accuracy: 0.3131
Epoch [7/500], Loss: 1.6184, Accuracy: 0.3151
Epoch [8/500], Loss: 1.6034, Accuracy: 0.3188
Epoch [9/500], Loss: 1.6020, Accuracy: 0.3193
Epoch [10/500], Loss: 1.5901, Accuracy: 0.3266
Epoch [11/500], Loss: 1.5912, Accuracy: 0.3296
Epoch [12/500], Loss: 1.5876, Accuracy: 0.3301
Epoch [13/500], Loss: 1.5837, Accuracy: 0.3326
Epoch [14/500], Loss: 1.5802, Accuracy: 0.3338
Epoch [15/500], Loss: 1.5771, Accuracy: 0.3355
Epoch [16/500], Loss: 1.5699, Accuracy: 0.3388
Epoch [17/500], Loss: 1.5696, Accuracy: 0.3387
Epoch [18/500], Loss: 1.5689, Accuracy: 0.3417
Epoch [19/500], Loss: 1.5691, Accuracy: 0.3403
Epoch [20/500], Loss: 1.5656, Accuracy: 0.3434
Epoch [21/500], Loss: 1.5599, Accuracy: 0.3440
Epoch [22/500], Loss: 

In [11]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(lstm_atn_model.parameters(), lr=0.001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.7773, Accuracy: 0.2380
Epoch [2/500], Loss: 1.7455, Accuracy: 0.2953
Epoch [3/500], Loss: 1.7124, Accuracy: 0.3151
Epoch [4/500], Loss: 1.6835, Accuracy: 0.3166
Epoch [5/500], Loss: 1.6585, Accuracy: 0.3166
Epoch [6/500], Loss: 1.6337, Accuracy: 0.3197
Epoch [7/500], Loss: 1.6189, Accuracy: 0.3215
Epoch [8/500], Loss: 1.6077, Accuracy: 0.3247
Epoch [9/500], Loss: 1.5956, Accuracy: 0.3252
Epoch [10/500], Loss: 1.5918, Accuracy: 0.3284
Epoch [11/500], Loss: 1.5875, Accuracy: 0.3319
Epoch [12/500], Loss: 1.5849, Accuracy: 0.3331
Epoch [13/500], Loss: 1.5786, Accuracy: 0.3375
Epoch [14/500], Loss: 1.5689, Accuracy: 0.3403
Epoch [15/500], Loss: 1.5714, Accuracy: 0.3413
Epoch [16/500], Loss: 1.5689, Accuracy: 0.3445
Epoch [17/500], Loss: 1.5650, Accuracy: 0.3430
Epoch [18/500], Loss: 1.5608, Accuracy: 0.3445
Epoch [19/500], Loss: 1.5584, Accuracy: 0.3474
Epoch [20/500], Loss: 1.5574, Accuracy: 0.3467
Epoch [21/500], Loss: 1.5584, Accuracy: 0.3487
Epoch [22/500], Loss: 

In [12]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(cnn_model.parameters(), lr=0.001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 1.8022, Accuracy: 0.1841
Epoch [2/500], Loss: 1.7688, Accuracy: 0.2019
Epoch [3/500], Loss: 1.7491, Accuracy: 0.2429
Epoch [4/500], Loss: 1.7279, Accuracy: 0.2621
Epoch [5/500], Loss: 1.7041, Accuracy: 0.2856
Epoch [6/500], Loss: 1.6848, Accuracy: 0.2930
Epoch [7/500], Loss: 1.6699, Accuracy: 0.2903
Epoch [8/500], Loss: 1.6517, Accuracy: 0.3014
Epoch [9/500], Loss: 1.6453, Accuracy: 0.3000
Epoch [10/500], Loss: 1.6404, Accuracy: 0.3046
Epoch [11/500], Loss: 1.6366, Accuracy: 0.2980
Epoch [12/500], Loss: 1.6270, Accuracy: 0.3077
Epoch [13/500], Loss: 1.6166, Accuracy: 0.3061
Epoch [14/500], Loss: 1.6149, Accuracy: 0.3128
Epoch [15/500], Loss: 1.6255, Accuracy: 0.3057
Epoch [16/500], Loss: 1.6127, Accuracy: 0.3126
Epoch [17/500], Loss: 1.6057, Accuracy: 0.3076
Epoch [18/500], Loss: 1.6048, Accuracy: 0.3123
Epoch [19/500], Loss: 1.6106, Accuracy: 0.3116
Epoch [20/500], Loss: 1.6056, Accuracy: 0.3126
Epoch [21/500], Loss: 1.6016, Accuracy: 0.3136
Epoch [22/500], Loss: 

In [13]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adagrad(cnn_atn_model.parameters(), lr=0.001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 1.7491, Accuracy: 0.2661
Epoch [2/500], Loss: 1.6718, Accuracy: 0.3168
Epoch [3/500], Loss: 1.6445, Accuracy: 0.3287
Epoch [4/500], Loss: 1.6264, Accuracy: 0.3358
Epoch [5/500], Loss: 1.6103, Accuracy: 0.3418
Epoch [6/500], Loss: 1.5989, Accuracy: 0.3447
Epoch [7/500], Loss: 1.5937, Accuracy: 0.3437
Epoch [8/500], Loss: 1.5849, Accuracy: 0.3568
Epoch [9/500], Loss: 1.5758, Accuracy: 0.3555
Epoch [10/500], Loss: 1.5721, Accuracy: 0.3585
Epoch [11/500], Loss: 1.5666, Accuracy: 0.3630
Epoch [12/500], Loss: 1.5636, Accuracy: 0.3603
Epoch [13/500], Loss: 1.5584, Accuracy: 0.3659
Epoch [14/500], Loss: 1.5592, Accuracy: 0.3633
Epoch [15/500], Loss: 1.5542, Accuracy: 0.3684
Epoch [16/500], Loss: 1.5505, Accuracy: 0.3702
Epoch [17/500], Loss: 1.5555, Accuracy: 0.3716
Epoch [18/500], Loss: 1.5551, Accuracy: 0.3704
Epoch [19/500], Loss: 1.5461, Accuracy: 0.3748
Epoch [20/500], Loss: 1.5411, Accuracy: 0.3699
Epoch [21/500], Loss: 1.5435, Accuracy: 0.3741
Epoch [22/500], Loss: 

In [14]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [15]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.7211, Test Accuracy: 0.2639


In [16]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:
Test Loss: 1.4622, Test Accuracy: 0.3835


In [17]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:
Test Loss: 1.4564, Test Accuracy: 0.3915


In [18]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:


Test Loss: 1.4881, Test Accuracy: 0.3714


In [19]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 1.4691, Test Accuracy: 0.3969
